In [1]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "11.0.8" 2020-07-14 LTS; Java(TM) SE Runtime Environment 18.9 (build 11.0.8+10-LTS); Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.8+10-LTS, mixed mode)
  Starting server from /usr/local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/95/21q0whwx4jn3944flf44yqmw0000gn/T/tmp8ekuop_y
  JVM stdout: /var/folders/95/21q0whwx4jn3944flf44yqmw0000gn/T/tmp8ekuop_y/h2o_kevin_started_from_python.out
  JVM stderr: /var/folders/95/21q0whwx4jn3944flf44yqmw0000gn/T/tmp8ekuop_y/h2o_kevin_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,"7 days, 14 hours and 42 minutes"
H2O_cluster_name:,H2O_from_python_kevin_i0eum6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [2]:
train = pd.read_csv('input/train.csv', index_col=0)
pred = pd.read_csv('input/predict.csv', index_col=0)

In [3]:
train.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
id,,,,,,,,,,
0,1.12,Premium,H,SI1,61.6,59.0,6.67,6.63,4.10,5363
1,1.14,Very Good,E,SI2,60.0,54.0,6.74,6.97,4.11,5593
2,0.90,Very Good,D,SI2,60.3,63.0,6.12,6.22,3.72,3534
3,0.71,Ideal,E,VS1,61.9,54.0,5.74,5.76,3.56,3212
4,0.34,Very Good,F,SI2,60.0,62.0,4.51,4.55,2.72,447


In [4]:
train = train[['carat','cut','color','clarity','x','y','z','price']]

In [5]:
h2train = h2o.H2OFrame(train)
h2pred = h2o.H2OFrame(pred)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
x = list(train.drop(columns = 'price').columns)
y = 'price'

In [7]:
automl = H2OAutoML(max_models = 50, seed = 1, max_runtime_secs = 2400, sort_metric = 'RMSE')
automl.train(x,y,training_frame=h2train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [8]:
leader_board = automl.leaderboard
leader_board.head()

model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201006_061151,532.887,283968,283968,270.757,0.0961854
StackedEnsemble_BestOfFamily_AutoML_20201006_061151,533.491,284613,284613,271.805,0.0966486
GBM_grid__1_AutoML_20201006_061151_model_1,538.098,289550,289550,278.218,0.101448
GBM_3_AutoML_20201006_061151,543.868,295792,295792,276.773,0.0993308
GBM_4_AutoML_20201006_061151,544.058,295999,295999,274.527,nan
GBM_2_AutoML_20201006_061151,544.994,297019,297019,279.613,0.102665
GBM_1_AutoML_20201006_061151,546.187,298320,298320,284.267,0.104568
GBM_5_AutoML_20201006_061151,562.363,316252,316252,278.721,0.100335
XGBoost_grid__1_AutoML_20201006_061151_model_1,563.047,317022,317022,283.493,0.0967316
XGBoost_grid__1_AutoML_20201006_061151_model_2,565.109,319348,319348,283.889,0.0958918


In [9]:
predictions2 = automl.leader.predict(h2pred)
predictions2.head()

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
746.457
6001.83
5886.2
1116.28
3226.74
592.603
1514.22
866.927
8429.4
671.873


In [10]:
pgr2 = predictions2.as_data_frame()
columns=['id','price']
pgr2 = pgr2.reset_index(level=0)
pgr2.columns = columns
pgr2.to_csv('last_try_2.csv', index = False)